In [5]:
import os
from langchain.llms import GooglePalm
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import GooglePalmEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA
key = os.environ['GOOGLE_API_KEY']
llm=GooglePalm(google_api_key=key,temperature=0.9)

In [6]:
embeddings = HuggingFaceEmbeddings()
vectordb=FAISS.load_local("faiss_index",embeddings)
retriever = vectordb.as_retriever()

In [7]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [8]:
chain_type_kwargs = {"prompt": PROMPT}
chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    input_key='query',
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [9]:
def ask(query):
    results = chain(query)
    if results:
        print("You Asked")
        print(results['query'])
        print("\n")
        
        print("Your Answer is")
        print(results['result'])
        print("\n")
        
        
        print("Answer is found from -")
        print(results['source_documents'])
        print("\n")
    else:
        print("No results found.")

In [11]:
ask('What is High Blood Pressure')

You Asked
What is High Blood Pressure


Your Answer is
?
answer: High blood pressure is a common disease in which blood flows through blood vessels (arteries) at higher than normal pressures. There are two main types of high blood pressure: primary and secondary high blood pressure. Primary, or essential, high blood pressure is the most common type of high blood pressure. This type of high blood pressure tends to develop over years as a person ages. Secondary high blood pressure is caused by another medical condition or use of certain medicines. This type usually resolves after the cause is treated or removed.


Answer is found from -
[Document(page_content=': 19\nquestion: What is (are) High Blood Pressure ?\nanswer: Blood pressure is the force of blood pushing against the walls of the blood vessels as the heart pumps blood. If your blood pressure rises and stays high over time, its called high blood pressure. High blood pressure is dangerous because it makes the heart work too hard, 

In [18]:
ask('What is Breast Cancer')

You Asked
What is Breast Cancer


Your Answer is
?
answer: Breast cancer is a disease in which certain cells in the breast become abnormal and multiply uncontrollably to form a tumor.


Answer is found from -
[Document(page_content=': 388\nquestion: What is (are) Breast Cancer ?\nanswer: Breast cancer occurs when a malignant tumor forms in the breast tissue. The cancer can be found in the breast itself or in the ducts and lymph nodes that surround the breast.\nsource: NIHSeniorHealth\nfocus_area: Breast Cancer', metadata={'source': 'What is (are) Breast Cancer ?', 'row': 388}), Document(page_content=': 10270\nquestion: What is (are) breast cancer ?\nanswer: Breast cancer is a disease in which certain cells in the breast become abnormal and multiply uncontrollably to form a tumor. Although breast cancer is much more common in women, this form of cancer can also develop in men. In both women and men, the most common form of breast cancer begins in cells lining the milk ducts (ductal canc

In [17]:
chain('what is high blood pressure')

{'query': 'what is high blood pressure',
 'result': '?\n High blood pressure is a common disease in which blood flows through blood vessels (arteries) at higher than normal pressures.',
 'source_documents': [Document(page_content=': 19\nquestion: What is (are) High Blood Pressure ?\nanswer: Blood pressure is the force of blood pushing against the walls of the blood vessels as the heart pumps blood. If your blood pressure rises and stays high over time, its called high blood pressure. High blood pressure is dangerous because it makes the heart work too hard, and the high force of the blood flow can harm arteries and organs such as the heart, kidneys, brain, and eyes.\nsource: NIHSeniorHealth\nfocus_area: High Blood Pressure', metadata={'source': 'What is (are) High Blood Pressure ?', 'row': 19}),
  Document(page_content=': 1661\nquestion: What is (are) High Blood Pressure ?\nanswer: Blood pressure is the force of your blood pushing against the walls of your arteries. Each time your hear

In [19]:
ask('What are the symptoms of Anemia in Chronic Kidney Disease')

You Asked
What are the symptoms of Anemia in Chronic Kidney Disease


Your Answer is
The signs and symptoms of anemia in someone with CKD may include
                
- weakness  - fatigue, or feeling tired  - headaches  - problems with concentration  - paleness  - dizziness  - difficulty breathing or shortness of breath  - chest pain
                
Anyone having difficulty breathing or with shortness of breath should seek immediate medical care. Anyone who has chest pain should call 911.


Answer is found from -
[Document(page_content=': 16248\nquestion: What are the symptoms of Anemia in Chronic Kidney Disease ?\nanswer: The signs and symptoms of anemia in someone with CKD may include\n                \n- weakness  - fatigue, or feeling tired  - headaches  - problems with concentration  - paleness  - dizziness  - difficulty breathing or shortness of breath  - chest pain\n                \nAnyone having difficulty breathing or with shortness of breath should seek immediate medical c